# Oblivious transfer

## Problem description:
Suppose that Alice wants to send two messages to Bob. Also, She wants to give the option to Bob to choose one of Her messages but She also wants to forbid the simultaneous reading of the two messages by Bob. That is, Bob can only read one of the messages depending on Bob's choice. Now, Bob doesn't want to make his choice visible for Alice. That is, only Bob can know what was his choice. To solve this problem we have to make the assumption that Alice and Bob will play by the rules no matter what. That is, Alice and Bob won't try to find the secret information of the other person.

## Example:
Supose that Alice wants to send one of the two strings "I like dogs" and "I like cats" to Bob. Bob made his choice to read the message 0. So Alice sends them both encrypted (to avoid Alice knowing the choice of Bob) and Bob will only be able to read the message 0 but not both. The steps they have to follow are the following.

Step zero: Imports and the declaration of two functions to convert from strings to bits and from bits to strings

In [44]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

First step: Alice chooses two messages. Additionally, She creates a merge of the messages, appending one on top of the other. Then, She sends them encrypted to Bob. The encryption works as follows: For each bit in the bit String, She choses a random number in the set {0,1}. If the number is 1, She encrypts the choosen bit and She leaves the bit without encryption otherwise. Notice that it would be impossible for Bob to know which bits were encrypted and which weren't. And also, it would be imposible for Bob to know what message was on top of the other for the messageMerge. 

In [56]:
#Alice cell, Bob can't see what's going in on here
m0 = tobits("I like dogs") 
m1 = tobits("I like cats")
messageMerge = m0+m1
Alice_bases = [randint(0,1) for x in range(len(messageMerge))]
qubits = list()
for i in range(len(Alice_bases)):
    mycircuit = QuantumCircuit(1,1)
    if(Alice_bases[i] == 0):
        if(messageMerge[i] == "1"):
            mycircuit.x(0)
    else:
        if(messageMerge[i] == "0"):
            mycircuit.h(0)
        else:
            mycircuit.x(0)
            mycircuit.h(0)
    qubits.append(mycircuit)

Second step: Bob receives Alice qubits and then He meassures them according to a random selection of basis. Also,  Bob chooses what message He will want to obtain from Alice. In the example below, He chooses the message 0.

In [57]:
#Bob cell, Alice can't see what's going in on here
Bob_bases = [randint(0,1) for x in range(176)]
backend = Aer.get_backend('qasm_simulator')
measurements = list()
choice = 0
for i in range(len(Bob_bases)):
    qubit = qubits[i]
    if(Bob_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))

Third step: Alice sends to Bob the basis She used to measure. Then Bob stores the indexes of the basis that correspond to the equal values between the ones sent by Alice and the random basis generated by Bob. Bob decides to store the set of the equal index in a set called I1 and the set of different index in a set called I0.

In [58]:
#Bob cell, Alice can't see what's going in on here
I0 = list()
I1 = list()
if(choice == 0):    
    for i in range(len(Alice_bases)):
        if(Alice_bases[i] == Bob_bases[i]):
            I0.append(i)
        else:
            I1.append(i)
else:
    for i in range(len(Alice_bases)):
        if(Alice_bases[i] == Bob_bases[i]):
            I1.append(i)
        else:
            I0.append(i)

Four step: Alice receives I1 and I0. Note that Alice doesn't know if I0 is the set of the equal basis or the set of distinct basis. And the same goes for I1. Then she prepares her final messages that She will send to Bob. The preparation consists in calculating a hash of the original messages in the indexes that Bob sent in I0 and I1. Then She applies an XOR between the bits of the hash and the bits of the original message.

In [59]:
#Alice cell, Bob can't see what's going in on here
x0 = list()
for x in I0:
    x0.append(messageMerge[x])
x1 = list()
for x in I1:
    x1.append(messageMerge[x])
fx0 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join(x0).encode('utf-8')).hexdigest())
fx1 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join(x1).encode('utf-8')).hexdigest())
s0 = ''
s1 = ''
for bit in range(len(m0)):
    s0 += str(int(fx0[bit]) ^ int(m0[bit]))
for bit in range(len(m1)):
    s1 += str(int(fx1[bit]) ^ int(m1[bit]))

Final step: Bob receives Alice messages S0 and S1. Then, He calculates the hash of the correct measurements that He obtained in the second step. Note that this hash is exactly the same as one of the two hashes that Alice have. Because both of them share the same string that created the hash. Then, He applies an XOR between the hash and the values that Alice sent him. Because only one of the two hashes were equals then Bob will be able to decipher one of Alice messages. And Alice didn't knew during the whole protocol the choice of Bob. So, the protocol is now completed and you can see in the output that it works quite well.

In [60]:
xB0 = list()
if(choice == 0):
    for x in I0:
        xB0.append(measurements[x])
else:
    for x in I1:
        xB0.append(measurements[x])
fxB0 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join([str(x) for x in xB0]).encode('utf-8')).hexdigest())
mB0 = bytearray()
if(choice == 0):
    for bit in range(len(s0)):
        mB0.append(int(fxB0[bit]) ^ int(s0[bit]))
    print("Alice message was: " + frombits(mB0))
else:
    for bit in range(len(s1)):
        mB0.append(int(fxB0[bit]) ^ int(s1[bit]))
    print("Alice message was: " + frombits(mB0))

Alice message was: I like dogs


## Why it doesn't work if Alice decides to cheat ?

If Alice entangles her qubits with another pair and she send them to Bob this could happend:
- Bob receives the entangle qubit and make a meassurement. At this point Alice know exactly which bits were meassured by Bob.
- Then Alice send the basis just like the protocol says.
- When alice receives I0 and I1, She knows exactly what was the desition of Bob because She got the Bob message from the entangle pair. The only thing she have to do is to check if the correct meassure qubits are in the message I0 or in the message I1.

## References:
Broadbent, A., Schaffner, C. Quantum cryptography beyond quantum key distribution. Des. Codes Cryptogr. 78, 351–382 (2016). https://doi.org/10.1007/s10623-015-0157-4